# Artificial Intelligence Spring 99 <img src = 'https://ece.ut.ac.ir/cict-theme/images/footer-logo.png' alt="Tehran-University-Logo" width="150" height="150" align="right">
## Project 3 : Text Processing and Bayesian Networks
### Dr. Hakimeh Fadaei
### By Omid Vaheb

## Introduction:
In this project, At first, we inspected data and cleaned it and normalize it for maximum accuracy. The next step was to implement a Bayesian network with 2 classes and a bag of word method to calculate words probabilities on the clean dataset. Then I implemented the network on a classification with 3 classes and used the model to classify some unknown data and got a good accuracy by checking with the original dataset in Kaggle.com.

## Question:
In this project we are asked to classify news category based on it's sgort description and headline. The solution is to implement Naive Bayes which is a simple Bayesian Network having one node of class and some other nodes representing features that are only connected to class node meaning that they are independent from each other. Another method used in this project is bag of words which is considering probbility of word regardless of their position in sentence. It means that probability of word to appear in any position in sentence is consistent. Now let's tell remaining explanations by analyzing code:

At first we import requiered libraries such as numpy and pandas for dataframe functions and re and nltk for data cleaning. Random and copy are used for minor parts.

In [1]:
import numpy as np
import pandas as pd
import re
import math
import copy
import random
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer 
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to C:\Users\Omid
[nltk_data]     Vaheb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Omid
[nltk_data]     Vaheb\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Omid
[nltk_data]     Vaheb\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

The first step is to laod dataset and inspect it a little. As you will see further it is dataset of news published on huffington post newspaper containing the author, date, headline, link and short description and the category of each news. It could be seen that there are some rows with no descriptions so I removed them due to project circumstances. The other thing that we can understand is that there are 3 type of news categories in the dataset: Business and Travel and Style and Beauty. Also the indexes in this project are important so they won't change throughout the project.

In [2]:
data = pd.read_csv('data.csv');

In [3]:
data.head(10)

,index,authors,category,date,headline,link,short_description
0,0,"Katherine LaGrave, ContributorTravel writer an...",TRAVEL,2014-05-07,"EccentriCities: Bingo Parties, Paella and Isla...",https://www.huffingtonpost.com/entry/eccentric...,Påskekrim is merely the tip of the proverbial ...
1,1,Ben Hallman,BUSINESS,2014-06-09,Lawyers Are Now The Driving Force Behind Mortg...,https://www.huffingtonpost.com/entry/mortgage-...,NaN
2,2,Jessica Misener,STYLE & BEAUTY,2012-03-12,Madonna 'Truth Or Dare' Shoe Line To Debut Thi...,https://www.huffingtonpost.com/entry/madonna-s...,"Madonna is slinking her way into footwear now,..."
3,3,"Victor and Mary, Contributor\n2Sense-LA.com",TRAVEL,2013-12-17,Sophistication and Serenity on the Las Vegas S...,https://www.huffingtonpost.com/entry/las-vegas...,But what if you're a 30-something couple that ...
4,4,"Emily Cohn, Contributor",BUSINESS,2015-03-19,It's Still Pretty Hard For Women To Get Free B...,https://www.huffingtonpost.com/entry/free-birt...,Obamacare was supposed to make birth control f...
5,5,Jessica Misener,STYLE & BEAUTY,2012-03-12,Madonna 'Truth Or Dare' Shoe Line To Debut Thi...,https://www.huffingtonpost.com/entry/madonna-s...,Madonna previously released a Truth or Dare fr...
6,6,Ellie Krupnick,STYLE & BEAUTY,2014-01-08,A Mean Girls Jewelry Line? That Is SO Fetch,https://www.huffingtonpost.com/entry/mean-girl...,"""Most all of our favorite movies have been cap..."
7,7,Simone Kitchens,STYLE & BEAUTY,2012-10-18,"Beauty Street Style: Stephanie Vanessa, Stylis...",https://www.huffingtonpost.com/entry/beauty-st...,"Take notes, ladies."
8,8,Ellie Krupnick,STYLE & BEAUTY,2013-01-31,Isabel Lucas Attempts Christian Dior Runway Lo...,https://www.huffingtonpost.com/entry/isabel-lu...,"Other times, something is lost in translation...."
9,9,"Budget Travel, Contributor\nYour source for va...",TRAVEL,2013-09-11,Warning: You'll Never Get These Items Through ...,https://www.huffingtonpost.com/entry/warning-y...,There are a few no-no's that U.S. customs will...


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22925 entries, 0 to 22924
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   index              22925 non-null  int64 
 1   authors            18523 non-null  object
 2   category           22925 non-null  object
 3   date               22925 non-null  object
 4   headline           22924 non-null  object
 5   link               22925 non-null  object
 6   short_description  21703 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.2+ MB


In [5]:
data.isnull().sum()

index                   0
authors              4402
category                0
date                    0
headline                1
link                    0
short_description    1222
dtype: int64

In [6]:
data = data[~data['short_description'].isnull()]

In [7]:
data.isnull().sum()

index                   0
authors              4182
category                0
date                    0
headline                0
link                    0
short_description       0
dtype: int64

As I said above there were 1222 rows without short description that i had removed them and filtered dataset keeping only index, category, headline and short description columns because the other ones weren't helpful for us. Using only short description would have resulted an acceptable answer but adding headline increased accuracy up to 10%.

In [8]:
dataframe = data.loc[:, ['index', 'category', 'headline', 'short_description']]

In [9]:
dataframe

,index,category,headline,short_description
0,0,TRAVEL,"EccentriCities: Bingo Parties, Paella and Isla...",Påskekrim is merely the tip of the proverbial ...
2,2,STYLE & BEAUTY,Madonna 'Truth Or Dare' Shoe Line To Debut Thi...,"Madonna is slinking her way into footwear now,..."
3,3,TRAVEL,Sophistication and Serenity on the Las Vegas S...,But what if you're a 30-something couple that ...
4,4,BUSINESS,It's Still Pretty Hard For Women To Get Free B...,Obamacare was supposed to make birth control f...
5,5,STYLE & BEAUTY,Madonna 'Truth Or Dare' Shoe Line To Debut Thi...,Madonna previously released a Truth or Dare fr...
...,...,...,...,...
22920,22920,STYLE & BEAUTY,Rihanna's Sweatpants Top Our Worst-Dressed Lis...,The theme for this week's worst-dressed list i...
22921,22921,STYLE & BEAUTY,Gisele Gives Birth To Baby Girl: Model Welcome...,Gisele finally confirms she's pregnant... by h...
22922,22922,TRAVEL,15 Golden Giveaways,"Travelzoo, a global Internet media company, is..."
22923,22923,TRAVEL,"Talking With Actor, Activist And Innkeeper Ric...","""We wanted to save it, so we bought it with th..."


Next step is cleaning data which i had written a function called cleanData for this purpose. It combines headline and short description each row and put it in text column and then lowercase all of the letters in the text and spliting them by non alphabetic characters. The next step was to convert word to their root format using nlkt librry methods stemming and lemmatizating. I used both of them nd got almost same answers (stemming was better a little bit so I used it). All of these actions are necessary in order t get an accurate answer. If we didn't lowercased all of our words there would be many versions of a word  with same value for us and in the state of probability calculation it will make our accuracy worse. A simple example for this would be the word money and Money which need to have same probability for business class but seperating them would split their probabilities and will lower the probability of this world. Another part of cleaning is converting word to their root. We could use stemming and lemmatizating. Stemming will only remove prefix and postixes, ing's , s's and things like this but won't change the format of verbs to their root like lemmatizating. In general it is a weaker method but it works better here. Another part of the data cleaning is removing word that are used alot in texts such as this, that, the and ... and won't help us in our classification method. I checked esults without lowercasing and root finding and each one decreased accuracy at least 10%.

In [10]:
stopWords = set(stopwords.words('english'))
def cleanData(row):
    row['text'] = row['headline'] + ' ' + row['short_description']
    row['text'] = row['text'].lower()
    wordTokens = re.split('\W', row['text'])
    filtered = [] 
    ps = PorterStemmer()
    for word in wordTokens:
        if word not in stopWords and not word == '' and not word.isdigit():
            root = ps.stem(word)
            filtered.append(root)
    row['text'] = filtered
    return row

In [11]:
dataframe = dataframe.apply(cleanData, axis = 'columns')
dataframe = dataframe.loc[:, ['index', 'category', 'text']]

In [12]:
dataframe

,index,category,text
0,0,TRAVEL,"[eccentr, bingo, parti, paella, island, hop, o..."
2,2,STYLE & BEAUTY,"[madonna, truth, dare, shoe, line, debut, fall..."
3,3,TRAVEL,"[sophist, seren, la, vega, strip, someth, coup..."
4,4,BUSINESS,"[still, pretti, hard, women, get, free, birth,..."
5,5,STYLE & BEAUTY,"[madonna, truth, dare, shoe, line, debut, fall..."
...,...,...,...
22920,22920,STYLE & BEAUTY,"[rihanna, sweatpant, top, worst, dress, list, ..."
22921,22921,STYLE & BEAUTY,"[gisel, give, birth, babi, girl, model, welcom..."
22922,22922,TRAVEL,"[golden, giveaway, travelzoo, global, internet..."
22923,22923,TRAVEL,"[talk, actor, activist, innkeep, richard, gere..."


### Classification with 2 classes:

In [13]:
dataframeTB = dataframe.loc[dataframe['category'] != 'STYLE & BEAUTY']

In [14]:
dataframeTB

,index,category,text
0,0,TRAVEL,"[eccentr, bingo, parti, paella, island, hop, o..."
3,3,TRAVEL,"[sophist, seren, la, vega, strip, someth, coup..."
4,4,BUSINESS,"[still, pretti, hard, women, get, free, birth,..."
9,9,TRAVEL,"[warn, never, get, item, custom, u, custom, co..."
11,11,TRAVEL,"[get, away, without, get, airplan, trip, easi,..."
...,...,...,...
22898,22898,TRAVEL,"[berlin, cruis, ship, northern, europ, scandin..."
22910,22910,BUSINESS,"[peac, corp, taught, manag, peopl, simultan, w..."
22914,22914,BUSINESS,"[bank, america, appear, flip, firearm, promis,..."
22922,22922,TRAVEL,"[golden, giveaway, travelzoo, global, internet..."


The next step is to create train and test groups. Train set consists of 80% of data and test will contain the rest. Selection of these sets are random and dataset eas splitted to 2 groups before slicing it to be ure that each contain 80% and 20% of both classes. The reason for that is to train our model to classify both classes and it needs a normal data for that but randomness of our selection could cause problem by assigning 70% of business class and 90% of travel class and it could make our classifier biased.

In [15]:
B = dataframeTB.loc[dataframeTB['category'] == 'BUSINESS']
T = dataframeTB.loc[dataframeTB['category'] == 'TRAVEL']

In [16]:
B['random_number'] = abs(np.random.random(len(B)))
trainB = B[B['random_number'] <= 0.8]
testB = B[B['random_number'] > 0.8]
T['random_number'] = abs(np.random.random(len(T)))
trainT = T[T['random_number'] <= 0.8]
testT = T[T['random_number'] > 0.8]
frames = [trainB, trainT]
train = pd.concat(frames)
frames = [testB, testT]
test = copy.deepcopy(pd.concat(frames))

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Before any operation we should define our probabilities: Posterior Probability( P(c|x) ) is the probability of category of news being c if words of the text are x. Likelihood is reverse of this probability meaning that it is probability of texts containing words equal to x if news category is c. Class prior probability is the probability of a news category publishing across all of the news published which is given to us ny an expert of this field. In this case I used the numbers of thisdataset on kaggle which had more than 200k of news data. Last one is predictor prior probability which is probability of a text cinsisting of set of word equal to x in general which we don't calculate in this project because we want to valculate maximum posterior probability between classes and this element is constant in all of classes and wouldn't affect the result.

In [17]:
PcB = math.log(5937 / 200853, 10) #according to huffington post's dataset
PcT = math.log(9887 / 200853, 10)
PcS = math.log(9649 / 200853, 10)

In [18]:
train

,index,category,text,random_number
15,15,BUSINESS,"[break, bad, final, ad, cost, much, report, fa...",0.787668
18,18,BUSINESS,"[cleveland, citi, council, vote, rais, minimum...",0.011517
23,23,BUSINESS,"[huge, california, ga, leak, come, end, februa...",0.175417
28,28,BUSINESS,"[women, busi, q, mari, pearson, founder, pears...",0.377325
43,43,BUSINESS,"[wall, street, lose, confid, exxon, mobil, low...",0.545618
...,...,...,...,...
22894,22894,TRAVEL,"[world, worst, airlin, name, airlin, name, tru...",0.430988
22897,22897,TRAVEL,"[exit, row, seat, get, much, everyon, love, li...",0.488723
22898,22898,TRAVEL,"[berlin, cruis, ship, northern, europ, scandin...",0.169522
22922,22922,TRAVEL,"[golden, giveaway, travelzoo, global, internet...",0.408819


In [19]:
test

,index,category,text,random_number
4,4,BUSINESS,"[still, pretti, hard, women, get, free, birth,...",0.879885
26,26,BUSINESS,"[nation, mortgag, settlement, miss, final, dea...",0.890291
53,53,BUSINESS,"[state, spend, least, educ, wall, st, base, u,...",0.821718
63,63,BUSINESS,"[elizabeth, warren, take, uber, lyft, gig, eco...",0.958009
66,66,BUSINESS,"[truth, statement, nonprofit, actual, busi, ma...",0.928863
...,...,...,...,...
22836,22836,TRAVEL,"[travel, loyalti, sale, mayb, time, rethink, m...",0.913720
22840,22840,TRAVEL,"[state, street, ballet, perfect, way, spend, h...",0.999828
22859,22859,TRAVEL,"[ryanair, announc, massiv, billion, order, boe...",0.970037
22861,22861,TRAVEL,"[top, place, escap, thanksgiv, ever, urg, esca...",0.814097


In [20]:
def extractWords(df):
    words = []
    for row in range(len(df)):
        words += df.iloc[row]['text']
    return words

The next step is to train the model by calculating probability of each word in each class by counting its frequecy in all of the texts with that class category. An important point here is to use and store probabilities using logarithm because they are such a small numbers that we cant store them.

In [21]:
trainBWords = extractWords(trainB)
n = len(set(trainBWords))
BProbabilities = dict( (l, math.log(trainBWords.count(l) / n, 10)) for l in set(trainBWords))
trainBWords = set(trainBWords)
trainTWords = extractWords(trainT)
m = len(set(trainTWords))
TProbabilities = dict( (l, math.log(trainTWords.count(l) / m, 10)) for l in set(trainTWords))
trainTWords = set(trainTWords)

This step is where we use probability of each word to classify the test data group. An important point here is to give a very low probability to a word when it is not available in trained word of a class vecause ignoring it is as same as giving it probability of 1. 

In [22]:
result = []
for row in range(len(test)):
    PBrow = PcB
    PTrow = PcT
    for word in test.iloc[row]['text']:
        if word in trainBWords and word in trainTWords:
            PBrow += BProbabilities[word]
            PTrow += TProbabilities[word]
        if word in trainBWords and not word in trainTWords:
            PBrow += BProbabilities[word]
            PTrow += -4
        if not word in trainBWords and word in trainTWords:
            PTrow += TProbabilities[word]
            PBrow += -4      
    if PBrow >= PTrow:
        result.append('BUSINESS')
    else:
        result.append('TRAVEL')
test['classified'] = result

In [23]:
numOfCorrectClassified = 0
correctClassifiedB = 0
correctClassifiedT = 0
detectedB = 0
detectedT = 0
for i in range(len(test)):
    if test.iloc[i]['classified'] == 'TRAVEL':
        detectedT += 1
    else:
        detectedB += 1
    if test.iloc[i]['classified'] == test.iloc[i]['category']:
        numOfCorrectClassified += 1
        if test.iloc[i]['classified'] == 'TRAVEL':
            correctClassifiedT += 1
        else:
            correctClassifiedB += 1

print('Recall of TRAVEL :', (correctClassifiedT / len(testT)) * 100, '    Recall of BUSINESS :', (correctClassifiedB / len(testB)) * 100)
print('Precision of TRAVEL :', (correctClassifiedT / detectedT) * 100, '  Precision of BUSINESS :', (correctClassifiedB / detectedB) * 100)
print('Accuracy :', (numOfCorrectClassified / len(test)) * 100 )

Recall of TRAVEL : 97.02233250620348     Recall of BUSINESS : 87.82894736842105
Precision of TRAVEL : 93.3731343283582   Precision of BUSINESS : 94.34628975265018
Accuracy : 93.70047543581617


We can see that all of evaluation parameters are higher than minimum accuracy needed. Now we implement steps above an aclassifier with 3 classes to classify instead of 2.

In [24]:
B = dataframe.loc[dataframe['category'] == 'BUSINESS']
T = dataframe.loc[dataframe['category'] == 'TRAVEL']
S = dataframe.loc[dataframe['category'] == 'STYLE & BEAUTY']

In [25]:
B['random_number'] = abs(np.random.random(len(B)))
trainB = B[B['random_number'] <= 0.8]
testB = B[B['random_number'] > 0.8]
T['random_number'] = abs(np.random.random(len(T)))
trainT = T[T['random_number'] <= 0.8]
testT = T[T['random_number'] > 0.8]
S['random_number'] = abs(np.random.random(len(S)))
trainS = S[S['random_number'] <= 0.8]
testS = S[S['random_number'] > 0.8]
frames = [trainB, trainT, trainS]
train = pd.concat(frames)
frames = [testB, testT, testS]
test = pd.concat(frames)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [26]:
trainBWords = extractWords(trainB)
n = len(set(trainBWords))
BProbabilities = dict( (l, math.log(trainBWords.count(l) / n, 10)) for l in set(trainBWords))
trainBWords = set(trainBWords)
trainTWords = extractWords(trainT)
m = len(set(trainTWords))
TProbabilities = dict( (l, math.log(trainTWords.count(l) / m, 10)) for l in set(trainTWords))
trainTWords = set(trainTWords)
trainSWords = extractWords(trainS)
k = len(set(trainSWords))
SProbabilities = dict( (l, math.log(trainSWords.count(l) / k, 10)) for l in set(trainSWords))
trainSWords = set(trainSWords)

In [27]:
result = []
for row in range(len(test)):
    PBrow = PcB
    PTrow = PcT
    PSrow = PcS
    for word in test.iloc[row]['text']:
        if word in trainBWords:
            PBrow += BProbabilities[word]
        else:
            PBrow += -4
        if word in trainTWords:
            PTrow += TProbabilities[word]
        else:
            PTrow += -4
        if word in trainSWords:
            PSrow += SProbabilities[word]
        else:
            PSrow += -4
    if PBrow >= PTrow and PBrow >= PSrow:
        result.append('BUSINESS')
    elif PTrow >= PBrow and PTrow >= PSrow:
        result.append('TRAVEL')
    else:
        result.append('STYLE & BEAUTY')
test['classified'] = result

In [28]:
test

,index,category,text,random_number,classified
18,18,BUSINESS,"[cleveland, citi, council, vote, rais, minimum...",0.900820,BUSINESS
75,75,BUSINESS,"[go, nowher, fast, mcdonald, forget, salad, wr...",0.837008,BUSINESS
77,77,BUSINESS,"[trendspot, still, matter, power, look, forwar...",0.920603,BUSINESS
82,82,BUSINESS,"[nike, middl, east, badass, new, ad, break, ba...",0.945046,STYLE & BEAUTY
87,87,BUSINESS,"[vulner, busi, good, thing, us, fundament, bel...",0.994631,BUSINESS
...,...,...,...,...,...
22904,22904,STYLE & BEAUTY,"[top, ten, right, list, make, happi, good, old...",0.813030,STYLE & BEAUTY
22907,22907,STYLE & BEAUTY,"[dear, mari, clair, magazin, instead, worri, l...",0.816937,STYLE & BEAUTY
22916,22916,STYLE & BEAUTY,"[new, season, new, makeup, trend, spring, desp...",0.959821,STYLE & BEAUTY
22920,22920,STYLE & BEAUTY,"[rihanna, sweatpant, top, worst, dress, list, ...",0.910603,STYLE & BEAUTY


In [29]:
numOfCorrectClassified = 0
correctClassifiedB = 0
correctClassifiedT = 0
correctClassifiedS = 0
detectedB = 0
detectedT = 0
detectedS = 0
PBAT = 0
PSAT = 0
PTAB = 0
PSAB = 0
PTAS = 0
PBAS = 0
for i in range(len(test)):
    if test.iloc[i]['classified'] == 'BUSINESS' and test.iloc[i]['category'] == "TRAVEL":
        PBAT += 1
    if test.iloc[i]['classified'] == 'STYLE & BEAUTY' and test.iloc[i]['category'] == "TRAVEL":
        PSAT += 1
    if test.iloc[i]['classified'] == 'TRAVEL' and test.iloc[i]['category'] == "BUSINESS":
        PTAB += 1
    if test.iloc[i]['classified'] == 'STYLE & BEAUTY' and test.iloc[i]['category'] == "BUSINESS":
        PSAB += 1
    if test.iloc[i]['classified'] == 'TRAVEL' and test.iloc[i]['category'] == "STYLE & BEAUTY":
        PTAS += 1
    if test.iloc[i]['classified'] == 'BUSINESS' and test.iloc[i]['category'] == "STYLE & BEAUTY":
        PBAS += 1
        
    if test.iloc[i]['classified'] == 'TRAVEL':
        detectedT += 1
    elif test.iloc[i]['classified'] == 'BUSINESS':
        detectedB += 1
    else:
        detectedS += 1
    if test.iloc[i]['classified'] == test.iloc[i]['category']:
        numOfCorrectClassified += 1
        if test.iloc[i]['classified'] == 'TRAVEL':
            correctClassifiedT += 1
        elif test.iloc[i]['classified'] == 'BUSINESS':
            correctClassifiedB += 1
        else:
            correctClassifiedS += 1
print('Recall of TRAVEL :', (correctClassifiedT / len(testT)) * 100, '    Recall of BUSINESS :',
      (correctClassifiedB / len(testB)) * 100, '    Recall of STYLE & BEAUTY :', (correctClassifiedS / len(testS)) * 100)
print('Precision of TRAVEL :', (correctClassifiedT / detectedT) * 100, '  Precision of BUSINESS :',
      (correctClassifiedB / detectedB) * 100,  '  Precision of STYLE & BEAUTY :', (correctClassifiedS / detectedS) * 100)
print('Accuracy :', (numOfCorrectClassified / len(test)) * 100 )

Recall of TRAVEL : 94.03697078115682     Recall of BUSINESS : 84.65553235908142     Recall of STYLE & BEAUTY : 96.0022844089092
Precision of TRAVEL : 92.49266862170089   Precision of BUSINESS : 92.68571428571428   Precision of STYLE & BEAUTY : 93.07862679955703
Accuracy : 92.77245782033744


 A confusion matrix is a table that is often used to describe the performance of a classifier on a set of test data for which the true values are known. It allows the visualization of the performance of an algorithm. Most performance measures are computed from the confusion matrix. The confusion matrix shows the ways in which your classification model is confused when it makes predictions. It gives us insight not only into the errors being made by a classifier but more importantly the types of errors that are being made. The first column of this table shows number of data that classifier classified as travel and each row of this column is actual number of data classified as travel, by each category. It means that the left top of table shows number of data that was actualy travel and classifier predicted it Travel. It is better for us to have higher numbers in diameter and lower in other cells. It could be seen that there are many data that are predicted style and beauty falsely.

In [30]:
print("   n =", len(test), "       Predicted Travel    Predicted Business  Predicted Style & B")
print("Actual Travel           ", correctClassifiedT, "               ", PBAT, "                ", PSAT)
print("Actual Business          ", PTAB, "                ", correctClassifiedB, "               ", PSAB)         
print("Actual Style & B         ", PTAS, "                ", PBAS, "                ", correctClassifiedS)

   n = 4386        Predicted Travel    Predicted Business  Predicted Style & B
Actual Travel            1577                 36                  64
Actual Business           86                  811                 61
Actual Style & B          42                  28                  1681


It could be seen that precision and recall of business class have a noticable difference and its reason is that number of business classes for training are lower than other classes so our word dictionary for this class is weaker and if a new word appear in our test it is more possible that it is available in the better-trained classes. Another thing is that when we add random data from current dataset data with higher probability is more probable to be chosen to add so its probability will increase and data with lower probability will be less probable. There are some ways to handle this like oversampling, undersampling, SMOTE and ... . The method we use here is oversampling in which we choose random rows from train data and add them to the train so that its population gets closer to other classes. This action will increase probability of each word meaning that weaker classes will be chosen more often. Adding about a 1000 rows almost equalize this evaluation metrics for all classes. Accuracy and recall don't change much but precision for business class gets better. An important point in oversapling is that train and test should be splitted before this becuse if you add from test to train it will discredit the result.

In [31]:
counts = dataframe['category'].value_counts()
counts

STYLE & BEAUTY    8674
TRAVEL            8461
BUSINESS          4568
Name: category, dtype: int64

In [32]:
B = dataframe.loc[dataframe['category'] == 'BUSINESS']
difference = int(((counts[0] + counts[1]) / 2) - counts[2]) - 3000
n = len(B)
t = len(B)
for c in range(difference):
    randomNumber = random.randint(0, n - 1)
    B.loc[t] = [B.iloc[randomNumber].iloc[0], B.iloc[randomNumber].iloc[1], B.iloc[randomNumber].iloc[2]]
    t += 1

D:\Anaconda\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
D:\Anaconda\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [38]:
B['random_number'] = abs(np.random.random(len(B)))
trainB = B[B['random_number'] <= 0.8]
testB = B[B['random_number'] > 0.8]
T['random_number'] = abs(np.random.random(len(T)))
trainT = T[T['random_number'] <= 0.8]
testT = T[T['random_number'] > 0.8]
S['random_number'] = abs(np.random.random(len(S)))
trainS = S[S['random_number'] <= 0.8]
testS = S[S['random_number'] > 0.8]
frames = [trainB, trainT, trainS]
train = pd.concat(frames)
frames = [testB, testT, testS]
test = pd.concat(frames)
trainBWords = extractWords(trainB)
n = len(set(trainBWords))
BProbabilities = dict( (l, math.log(trainBWords.count(l) / n, 10)) for l in set(trainBWords))
trainBWords = set(trainBWords)
trainTWords = extractWords(trainT)
m = len(set(trainTWords))
TProbabilities = dict( (l, math.log(trainTWords.count(l) / m, 10)) for l in set(trainTWords))
trainTWords = set(trainTWords)
trainSWords = extractWords(trainS)
m = len(set(trainSWords))
SProbabilities = dict( (l, math.log(trainSWords.count(l) / m, 10)) for l in set(trainSWords))
trainSWords = set(trainSWords)
result = []
for row in range(len(test)):
    PBrow = PcB
    PTrow = PcT
    PSrow = PcS
    for word in test.iloc[row]['text']:
        if word in trainBWords:
            PBrow += BProbabilities[word]
        else:
            PBrow += -4
        if word in trainTWords:
            PTrow += TProbabilities[word]
        else:
            PTrow += -4
        if word in trainSWords:
            PSrow += SProbabilities[word]
        else:
            PSrow += -4
    if PBrow >= PTrow and PBrow >= PSrow:
        result.append('BUSINESS')
    elif PTrow >= PBrow and PTrow >= PSrow:
        result.append('TRAVEL')
    else:
        result.append('STYLE & BEAUTY')
test['classified'] = result
numOfCorrectClassified = 0
correctClassifiedB = 0
correctClassifiedT = 0
correctClassifiedS = 0
detectedB = 0
detectedT = 0
detectedS = 0
PBAT = 0
PSAT = 0
PTAB = 0
PSAB = 0
PTAS = 0
PBAS = 0
for i in range(len(test)):
    if test.iloc[i]['classified'] == 'BUSINESS' and test.iloc[i]['category'] == "TRAVEL":
        PBAT += 1
    if test.iloc[i]['classified'] == 'STYLE & BEAUTY' and test.iloc[i]['category'] == "TRAVEL":
        PSAT += 1
    if test.iloc[i]['classified'] == 'TRAVEL' and test.iloc[i]['category'] == "BUSINESS":
        PTAB += 1
    if test.iloc[i]['classified'] == 'STYLE & BEAUTY' and test.iloc[i]['category'] == "BUSINESS":
        PSAB += 1
    if test.iloc[i]['classified'] == 'TRAVEL' and test.iloc[i]['category'] == "STYLE & BEAUTY":
        PTAS += 1
    if test.iloc[i]['classified'] == 'BUSINESS' and test.iloc[i]['category'] == "STYLE & BEAUTY":
        PBAS += 1
        
    if test.iloc[i]['classified'] == 'TRAVEL':
        detectedT += 1
    elif test.iloc[i]['classified'] == 'BUSINESS':
        detectedB += 1
    else:
        detectedS += 1
    if test.iloc[i]['classified'] == test.iloc[i]['category']:
        numOfCorrectClassified += 1
        if test.iloc[i]['classified'] == 'TRAVEL':
            correctClassifiedT += 1
        elif test.iloc[i]['classified'] == 'BUSINESS':
            correctClassifiedB += 1
        else:
            correctClassifiedS += 1
print('Recall of TRAVEL :', (correctClassifiedT / len(testT)) * 100, '    Recall of BUSINESS :',
      (correctClassifiedB / len(testB)) * 100, '    Recall of STYLE & BEAUTY :', (correctClassifiedS / len(testS)) * 100)
print('Precision of TRAVEL :', (correctClassifiedT / detectedT) * 100, '  Precision of BUSINESS :',
      (correctClassifiedB / detectedB) * 100,  '  Precision of STYLE & BEAUTY :', (correctClassifiedS / detectedS) * 100)
print('Accuracy :', (numOfCorrectClassified / len(test)) * 100 )
print("   n =", len(test), "       Predicted Travel    Predicted Business  Predicted Style & B")
print("Actual Travel           ", correctClassifiedT, "               ", PBAT, "                ", PSAT)
print("Actual Business          ", PTAB, "                ", correctClassifiedB, "               ", PSAB)         
print("Actual Style & B         ", PTAS, "                ", PBAS, "                ", correctClassifiedS)

D:\Anaconda\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

Recall of TRAVEL : 91.47869674185463     Recall of BUSINESS : 90.83409715857012     Recall of STYLE & BEAUTY : 94.53357100415924
Precision of TRAVEL : 93.89067524115757   Precision of BUSINESS : 88.87892376681614   Precision of STYLE & BEAUTY : 93.58823529411765
Accuracy : 92.49427917620137
   n = 4370        Predicted Travel    Predicted Business  Predicted Style & B
Actual Travel            1460                 74                  62
Actual Business           53                  991                 47
Actual Style & B          42                  50                  1591


In [34]:
print(len(B), len(T), len(S))

5363 8461 8674


The final step is to classify unknown data give to us in test.csv file. After loading the dataset we clean data and split texts to words and classify news based on their words and model trained in last part. I checked the result written in output.csv with the correct data from kaggle.com and got 95% accuracy.

In [35]:
unclassified = pd.read_csv('test.csv')
stopWords = set(stopwords.words('english'))
def cleanData(row):
    if row['short_description'] == None:
        row['text'] = row['headline'] + ' ' + row['short_description']
    else:
        row['text'] = row['headline']
    row['text'] = row['text'].lower()
    wordTokens = re.split('\W', row['text'])
    filtered = [] 
    ps = PorterStemmer()
    for word in wordTokens:
        if word not in stopWords and not word == '' and not word.isdigit():
            root = ps.stem(word)
            filtered.append(root)
    row['text'] = filtered
    return row

In [36]:
unclassified = unclassified.apply(cleanData, axis = 'columns')
unclassified = unclassified.loc[:, ['index', 'text']]
unclassified

,index,text
0,0,"[kate, middleton, one, two, style, win, new, z..."
1,1,"[instagram, local, len, seri, featur, insid, l..."
2,2,"[go, thanksgiv]"
3,3,"[retail, hire, employe, holiday]"
4,4,"[get, flat, iron, wave, minut, video]"
...,...,...
2543,2543,"[state, line, fiscal, cliff, debat]"
2544,2544,"[multipurpos, travel, essenti, need, next, adv..."
2545,2545,"[santa, claus, unemploy, holiday, season, video]"
2546,2546,"[like, food, tv, train, women, leader, culinar..."


In [37]:
result = []
for row in range(len(unclassified)):
    PBrow = PcB
    PTrow = PcT
    PSrow = PcS
    for word in unclassified.iloc[row]['text']:

        if word in trainBWords:
            PBrow += BProbabilities[word]
        else:
            PBrow += -4
        if word in trainTWords:
            PTrow += TProbabilities[word]
        else:
            PTrow += -4
        if word in trainSWords:
            PSrow += SProbabilities[word]
        else:
            PSrow += -4
    if PBrow >= PTrow and PBrow >= PSrow:
        result.append('BUSINESS')
    elif PTrow >= PBrow and PTrow >= PSrow:
        result.append('TRAVEL')
    else:
        result.append('STYLE & BEAUTY')
unclassified['classified'] = result
unclassified = unclassified.loc[:, ['index', 'classified']]
unclassified.to_csv('output.csv', index = False)

### Questions:
#### 1- 
I answered the first question in the report. Although lemmatization is more advaned they both result in a same way.
#### 2- 
The tf-idf weight a statistical measure used to evaluate how important a word is to a document. The importance increases proportionally to the number of times a word appears in the document but is offset by the frequency of the word in the dataset. This weight is computed by finding product of term frequency (tf) and inverse document frequency. Term Frequency indicates the number of occurences of a particular term t in document d. We can see that as a term appears more in the document it becomes more important, which is logical. We need to normalize it by by dividing it by total number of words. The idf of a term is the number of documents in the dataset divided by the document frequency of a term. Term frequency is the occurrence count of a term in one particular document only; while document frequency is the number of different documents the term appears in, so it depends on the whole dataset. It’s expected that the more frequent term to be considered less important, but the factor (most probably integers) seems too harsh. Therefore, we take the logarithm (with base 2 ) of the inverse document frequencies. The last step is to calculate product of these two measures for each word. In order to use this measure we could have used it instead of likelihood that we calculated in previous steps.
#### 3-
Prescision is an important evaluation measure but it is not enough and focusing on just this measure will cause problem; Our model could be riskless and classify only items that is sure about them and this will decrease accuracy and recall. For example if we have a classifier for ham and spam detection and we choose only one email as spam that has the highest probability we will probably get precision of 100% but accuracy and recall will be very low. If precision for all classes of classifier are high then our classifier has worked realy well because we have classified all of data and did it flawless so accuracy and recall will also be high.
#### 4-
If a word appears in train in one class only, in the step that we want calculate probability of each class for test data the class containing this word will get a high probability and other classes will get a small number( 10^-4) that will make them less probable and data will probably be classified as category containing spoken word.